In [1]:
import time
import copy
import random

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from graph_tree_link_module import *


In [2]:
raw = pd.read_csv(r'../Data/train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [3]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)
data = data[:2000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(2000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [4]:
data_min = scaler.data_min_
data_max = scaler.data_max_

# print(data_min, data_max)

In [5]:
def test_query_generator():
    return [random.uniform(data_min[i], data_max[i]) for i in range(len(data_max))]

In [6]:
print(test_query_generator())

[np.float64(31.016805846839198), np.float64(0.7291278423356826), np.float64(0.24147426922388437), np.float64(5.331519915598523), np.float64(-2.8552650708117753), np.float64(0.6525964137617971), np.float64(0.10845424067186216), np.float64(0.9645368391954534), np.float64(0.2232042816584578), np.float64(0.6178356824439832), np.float64(0.3009444075232025), np.float64(163.22372485514902), np.float64(871416.2469655385), np.float64(1.7102007994855901)]


In [7]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end_time = time.perf_counter()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time} seconds")

In [8]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, X_maxmin=[scaler.data_max_,scaler.data_min_])
# test_tree.show_root()
with Timer():
    test_tree.build_tree(min_threshold_for_clustering=200)


---------------------------------- level 0 no of node 1 ----------------------------------------
queue:  [0]
cur node 0
no. of cur node data 2000
optimal K 2


---------------------------------- level 1 no of node 2 ----------------------------------------
queue:  [1, 2]
cur node 1
no. of cur node data 719
optimal K 4

cur node 2
no. of cur node data 1281
optimal K 2


---------------------------------- level 2 no of node 6 ----------------------------------------
queue:  [3, 4, 5, 6, 7, 8]
cur node 3
no. of cur node data 238
optimal K 2

cur node 4
no. of cur node data 277
optimal K 3

cur node 5
no. of cur node data 78
added to Popularity min
added to danceability min
added to energy max
added to key max
added to loudness max
added to mode min
added to speechiness min
added to acousticness min
added to instrumentalness max
added to liveness min
added to valence min
added to tempo max
added to duration_in min/ms max
added to time_signature min
cur node 6
no. of cur node data 126
adde

In [9]:
# import pickle

# with open("graph_tree.pkl", 'wb') as f:
#     pickle.dump(test_tree, f)

In [10]:
t = 200000
while t>100:
    seaching = test_query_generator()
    with Timer():
        cur = test_tree.search(seaching)
    t=cur.id
    print(cur.id)

Elapsed time: 0.0005437999789137393 seconds
14


In [11]:
a = test_tree._get_closest_node_within_lv(seaching, cur)
print([n.id for n in a])
print(cur.down_lv)

will return a sorted list of leaf node
[14]
poped 14
cal True False
[]
[14]
[]


In [12]:
print(calculation.l1(seaching, a[-1].center))
print(calculation.l1(seaching, a[0].center))

620402.1940225144
620402.1940225144


In [13]:
test_tree.print_tree(test_tree.root)

└── 0
    ├── 1
    │   ├── 3
    │   │   ├── 9
    │   │   └── 10
    │   ├── 4
    │   │   ├── 11
    │   │   ├── 12
    │   │   └── 13
    │   ├── 5
    │   └── 6
    └── 2
        ├── 7
        │   ├── 14
        │   └── 15
        │       ├── 19
        │       └── 20
        └── 8
            ├── 16
            │   ├── 21
            │   └── 22
            ├── 17
            │   ├── 23
            │   │   ├── 25
            │   │   ├── 26
            │   │   └── 27
            │   └── 24
            │       ├── 28
            │       └── 29
            └── 18


In [14]:
res = test_tree.reflection_search(np.mean(normalized_data[:100], axis=0), test_tree.root)

TypeError: graph_tree.reflection_search() missing 1 required positional argument: 'degree'

In [ ]:
res

In [ ]:
print(normalized_data.shape)
print(np.mean(normalized_data[:100], axis=0))

print(test_tree.root.center)

In [ ]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

In [ ]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))